In [106]:
# %load query.py
import numpy as np
from astropy import units as u
from astropy.io import ascii
from astroquery.xmatch import XMatch
import requests

In [2]:
# %load query.py
import numpy as np
import glob

from astropy import units as u
from astropy.io import ascii
from astroquery.xmatch import XMatch
from astropy.table import Table
from astropy.coordinates import SkyCoord, ICRS, Galactic, Angle

In [3]:
files = glob.glob('../IPTA_DR1/*/*.par')

In [107]:
RAs = []
DECs = []
Names = []
pmra = []
pmdec = []
pepoch = []
posepoch = []
dmepoch = []
dm = []

for file in files:
    d = {}
    with open(file) as f:
        for line in f:
            key = line.split()[0]
            value = line.split()[1]
            d[key] = value
            
    try:
        RAs.append(Angle(d['RAJ'], u.hourangle).degree)
        #print(d['PSRJ'], d['DECJ'])
        DECs.append(Angle(d['DECJ'], u.degree).value)
        Names.append(d['PSRJ'])
        pmra.append(d['PMRA'])
        pmdec.append(d['PMDEC'])
        pepoch.append(d['PEPOCH'])
        posepoch.append(d['POSEPOCH'])
        dmepoch.append(d['DMEPOCH'])
        dm.append(d['DM'])
        
    except KeyError:
        coord = Galactic(l=float(d['ELONG'])*u.deg, b=float(d['ELAT'])*u.deg, pm_l_cosb = float(d['PMELONG'])*np.cos(float(d['ELAT'])*np.pi/180.)*u.mas/u.yr, pm_b=float(d['PMELAT'])*u.mas/u.yr)
        icrscoord = coord.transform_to(ICRS)
        #print(icrscoord)
        #print(d['PSRJ'])
        RAs.append(icrscoord.ra.to(u.deg).value)
        DECs.append(icrscoord.dec.to(u.deg).value)
        Names.append(d['PSRJ'])
        pmra.append(icrscoord.pm_ra_cosdec/np.cos(icrscoord.dec.to(u.rad)))
        pmdec.append(icrscoord.pm_dec)
        pepoch.append(d['PEPOCH'])
        posepoch.append(d['POSEPOCH'])
        dmepoch.append(d['DMEPOCH'])
        dm.append(d['DM'])        
        #coord = SkyCoord(d[''])
        
        continue

t = Table([Names, RAs, DECs, pmra, pmdec, pepoch, posepoch, dmepoch, dm], names=['names', 'ra', 'dec', 'pmra', 'pmdec', 'dm', 'pepoch', 'posepoch', 'dmepoch'])        
tmin = Table([RAs, DECs], names=['ra', 'dec'])


In [24]:
raAng = [Angle(ra, u.hourangle).degree for ra in RAs]
print(raAng)

[7.614285091249999, 8.590975955416665, 34.52648865833333, 69.31623406223791, 92.55664775541666, 93.43323197999999, 95.34214323999998, 107.97578802208332, 117.78814720749997, 135.18313787124998, 153.13932304124995, 156.16114741416666, 161.45911275499998, 223.94987448041664, 240.21626390166665, 240.89865625, 250.06977012833332, 250.90900619013118, 258.4563882341666, 260.2729079616667, 262.5902839079166, 263.1990280079167, 264.7248598870833, 266.12253142416665, 267.88622164541664, 270.46280559375, 270.52223016333335, 271.0880545658333, 276.13336630624997, 280.9219246283333, 283.48882820999995, 284.40162750624995, 287.4476398970833, 287.540422595, 287.9800194858333, 287.9553430220833, 289.70013800916666, 294.91067175291664, 298.8661489208333, 302.69133598625, 304.88308706583325, 308.3646423708333, 321.1826992370833, 322.3448623666667, 326.4602524958333, 337.4620225720833, 349.2884858925, 350.59306347791664]


In [86]:
t.write('ipta_data.csv', format='csv', overwrite=True)
tmin.write('ipta_data_nonames.csv', format='csv', overwrite=True)

In [104]:
r = requests.post(
'http://cdsxmatch.u-strasbg.fr/xmatch/api/v1/sync',
data={'request': 'xmatch', 'distMaxArcsec': 0.5, 'RESPONSEFORMAT': 'csv',
'cat2': 'vizier:I/337/gaia', 'colRA1': 'ra', 'colDec1': 'dec'},
files={'cat1': open('ipta_data.csv', 'r')})

In [105]:
print(r.text)

angDist,names,ra,dec,pmra,pmdec,dm,pepoch,posepoch,dmepoch,ra_ep2000,dec_ep2000,errHalfMaj,errHalfMin,errPosAng,ra,dec,source_id,ref_epoch,ra_error,dec_error,parallax,parallax_error,pmra,pmra_error,pmdec,pmdec_error,ra_dec_corr,duplicated_source,phot_g_n_obs,phot_g_mean_flux,phot_g_mean_flux_error,phot_g_mean_mag,phot_variable_flag,l,b
0.140141,J1012+5307,153.13932304124995,53.1173056,2.6237122562025093,-25.458808920674365,55000,55000,55000,9.021803950562022,153.1393295862,53.1172668707,0.001,0.000,0,153.1393295862,53.1172668707,851610857095609984,2015.0,0.395,0.614,,,,,,,0.058,0,148,2.43759e+02,1.93173e+00,19.557,NOT_AVAILABLE,160.3471938161,50.8578532615
0.379287,J1024-0719,156.16114741416666,-7.322064986111111,-35.30492690451154,-48.2439401720538,55000,55000,55000,6.486302522549919,156.1610974393,-7.3221579554,0.011,0.007,0,156.1610974393,-7.3221579554,3775277868091720192,2015.0,6.547,11.160,,,,,,,-0.857,0,33,3.91727e+02,4.93626e+00,19.042,NOT_AVAILABLE,251.7016017396,40.5153956229


In [103]:
h = open('ipta_crossmatch_1arcsecond.csv', 'w')
h.write(r.text)
h.close()